In [3]:
import requests
from urllib.parse import urljoin
import urllib
from bs4 import BeautifulSoup

In [4]:
def getAllUrl(url):
    urlList = []
    print(url)
    try:
        page = requests.get(url).text
        soup = BeautifulSoup(page)
    except:
        print("Nope")
        return []
    try:
        for anchor in soup.find_all("a"):
            if "href" in anchor.attrs:
                if "download" not in anchor["href"]:
                    if "https://" not in anchor["href"] and anchor["href"][0] == '/':
                        urlList.append("https://www.millenaire3.com"+anchor["href"])
                    elif "https://www.millenaire3.com" in anchor["href"]:
                        urlList.append(anchor["href"])

        return(urlList)
    except:
        print("http error")
        return(None)

In [5]:

if __name__ == "__main__":
    urls = getAllUrl('https://www.millenaire3.com')
    i = 0
    while i < len(urls):
        urlList = getAllUrl(urls[i])
        if urlList != None:
            urls = urls + urlList
            urls = list(set(urls))
        i += 1
        print(i, len(urls))

with open("M3urls.txt", "w", encoding="utf-8") as f:
    for url in urls:
        f.writelines(url)


https://www.millenaire3.com
https://www.millenaire3.com/
37
https://www.millenaire3.com/actualites-a-la-une/veillem3-la-famille-digitale-pres-des-yeux-loin-du-caeur
41
https://www.millenaire3.com/actualites-a-la-une/veillem3-la-famille-digitale-pres-des-yeux-loin-du-caeur
41
https://www.millenaire3.com/ressources/2023/metabolisme-de-l-economie-3-4-quelle-responsabilite-et-quels-enjeux-pour-l-europe
57
https://www.millenaire3.com/ressources/2023/metabolisme-de-l-economie-2-4-transformer-le-metabolisme-un-defi-sans-precedent
59
https://www.millenaire3.com/actualites-a-la-une/veillem3-la-famille-digitale-pres-des-yeux-loin-du-caeur
59
https://www.millenaire3.com/actualites-a-la-une/veillem3-la-famille-digitale-pres-des-yeux-loin-du-caeur
59
https://www.millenaire3.com/ressources/2023/metabolisme-de-l-economie-3-4-quelle-responsabilite-et-quels-enjeux-pour-l-europe
59
https://www.millenaire3.com/ressources/2023/pratiques-et-preferences-culturelles-des-jeunes-metropolitains
69
https://www.m